In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cricket-diss-2/dissertation dataset.xlsx


In [2]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812366 sha256=cd92f5818e4f064fd60eb0241ffc14e5820f9a8ecf83d6ce696bf381c69cee53
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
import pandas as pd

# Initialize a Spark session
spark = SparkSession.builder.appName("ODIWorldCupAnalysis").getOrCreate()

# Load the datasets
file_path = '/kaggle/input/cricket-diss-2/dissertation dataset.xlsx' 
data = pd.read_excel(file_path, sheet_name=None)  
# Load specific sheets into DataFrames
odi_2011_stats = data['2011 odi world cup stats']
odi_2023_stats = data['2023 odi world cup stats']

# Preview the first few rows of each DataFrame
print(odi_2023_stats.head())
print(odi_2011_stats.head())
# Preview the last few rows of each DataFrame
print(odi_2023_stats.tail())
print(odi_2011_stats.tail())





Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/23 10:13:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


   Match numb+C7+A1:G40        Team 1        Team 2  Team 1 runs  \
0                     1       England   New Zealand          282   
1                     2      Pakistan   Netherlands          286   
2                     4  South Africa    Sri Lanka           428   
3                     5     Australia         India          199   
4                     6   New Zealand  Netherlands           322   

   Team 2 runs   Team 1 batting average  Team 2 batting average  \
0           283                    31.3                   282.0   
1           205                    28.6                    20.5   
2           326                    85.6                    32.6   
3           201                    19.9                    50.3   
4           223                    46.0                    22.3   

   Team 1 batting run rate  team 2 batting run rate  Team 1 catches taken  \
0                     5.64                     7.79                     1   
1                     5.83        

In [4]:
# Clean column names (remove any trailing spaces)
odi_2011_stats.columns = odi_2011_stats.columns.str.strip()
odi_2023_stats.columns = odi_2023_stats.columns.str.strip()

# Identify non-numeric values in 'Team 2 batting average'
non_numeric_values = odi_2011_stats[~odi_2011_stats['Team 2 batting average'].apply(pd.to_numeric, errors='coerce').notnull()]

print(non_numeric_values['Team 2 batting average'])

# Replace non-numeric values with NaN
odi_2011_stats['Team 2 batting average'] = pd.to_numeric(odi_2011_stats['Team 2 batting average'], errors='coerce')

# Verify the change
print(odi_2011_stats['Team 2 batting average'].dtypes)
print(odi_2011_stats['Team 2 batting average'].isnull().sum())  

17    N/A (231/0)
Name: Team 2 batting average, dtype: object
float64
1


In [5]:
print(odi_2023_stats.describe())
print(odi_2011_stats.describe())

       Match numb+C7+A1:G40  Team 1 runs  Team 2 runs  Team 1 batting average  \
count             39.000000    39.000000    39.000000               39.000000   
mean              24.820513   298.923077   223.333333               40.815385   
std               14.195759    75.033812    71.512359               23.063055   
min                1.000000   156.000000    55.000000               15.600000   
25%               13.000000   242.500000   178.000000               25.800000   
50%               25.000000   286.000000   223.000000               32.000000   
75%               36.500000   360.500000   267.000000               45.300000   
max               48.000000   428.000000   383.000000              102.500000   

       Team 2 batting average  Team 1 batting run rate  \
count               39.000000                39.000000   
mean                50.623077                 6.107949   
std                 55.514950                 1.359824   
min                  5.500000         

In [7]:

# Convert Pandas DataFrames to Spark DataFrames
spark_df_2011 = spark.createDataFrame(odi_2011_stats)
spark_df_2023 = spark.createDataFrame(odi_2023_stats)

# Verify the conversion
spark_df_2011.printSchema()
spark_df_2023.printSchema()



root
 |-- Match number: long (nullable = true)
 |-- Team 1: string (nullable = true)
 |-- Team 2: string (nullable = true)
 |-- Team 1 runs: long (nullable = true)
 |-- Team 2 runs: long (nullable = true)
 |-- Team 1 batting average: double (nullable = true)
 |-- Team 2 batting average: double (nullable = true)
 |-- Team 1 batting run rate: double (nullable = true)
 |-- team 2 batting run rate: double (nullable = true)
 |-- Team 1 catches taken: long (nullable = true)
 |-- Team 2 catches taken: long (nullable = true)
 |-- Team 1 wickets taken by run out: long (nullable = true)
 |-- Team 2 wickets taken by run out: long (nullable = true)
 |-- Team 1 - runs scored by extras: long (nullable = true)
 |-- Team 2 - runs scored by extras: long (nullable = true)
 |-- Team 1 50's: long (nullable = true)
 |-- team 2 50's: long (nullable = true)
 |-- team 1 100's: long (nullable = true)
 |-- team 2 100's: long (nullable = true)
 |-- team 1 % of overs bowled by pace: double (nullable = true)
 |-- 

In [8]:
# Create temporary views
spark_df_2011.createOrReplaceTempView("odi_2011_stats")
spark_df_2023.createOrReplaceTempView("odi_2023_stats")

24/08/23 10:54:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:

from scipy.stats import ttest_ind

# Extract data
query_2011 = """
SELECT `Team 2 wickets taken by run out`
FROM odi_2011_stats
"""
team1_batting_run_rate_2011 = spark.sql(query_2011).toPandas()

# Extract data
query_2023 = """
SELECT `Team 2 wickets taken by run out`
FROM odi_2023_stats
"""
team1_batting_run_rate_2023 = spark.sql(query_2023).toPandas()

# Perform the t-test
t_stat, p_val = ttest_ind(team1_batting_run_rate_2011['Team 2 wickets taken by run out'], team1_batting_run_rate_2023['Team 2 wickets taken by run out'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

T-Statistic: 1.689066839004371, P-Value: 0.10122236110158525


In [10]:
# Extract the data for non-knockout matches
query_non_knockout = """
SELECT (`Team 2 - runs scored by extras`) 
FROM odi_2011_stats
WHERE `knockout yes or no` = 'no'
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT (`Team 2 - runs scored by extras`) 
FROM odi_2011_stats
WHERE `knockout yes or no` = 'yes'
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['Team 2 - runs scored by extras'], knockout_df['Team 2 - runs scored by extras'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

Index(['Team 2 - runs scored by extras'], dtype='object')
Index(['Team 2 - runs scored by extras'], dtype='object')
T-Statistic: 0.23945997393227003, P-Value: 0.8145010973982204


In [12]:
from pyspark.sql import SparkSession
import pandas as pd
from scipy.stats import ttest_ind

# Extract the data for the performance metric (e.g., Team 1 batting run rate) in 2011
query_2011 = """
SELECT `Team 1 batting average`
FROM odi_2011_stats
"""
team1_batting_run_rate_2011 = spark.sql(query_2011).toPandas()

# Extract the data for the performance metric (e.g., Team 1 batting run rate) in 2023
query_2023 = """
SELECT `Team 1 batting average`
FROM odi_2023_stats
"""
team1_batting_run_rate_2023 = spark.sql(query_2023).toPandas()

# Perform the t-test
t_stat, p_val = ttest_ind(team1_batting_run_rate_2011['Team 1 batting average'], team1_batting_run_rate_2023['Team 1 batting average'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

T-Statistic: -1.010790584135412, P-Value: 0.3170794974565816


In [13]:
from pyspark.sql import SparkSession
import pandas as pd
from scipy.stats import ttest_ind

# Initialize Spark session
spark = SparkSession.builder \
    .appName("ODI Analysis") \
    .getOrCreate()

# Extract the data for non-knockout matches
query_non_knockout = """
SELECT (`Team 1 batting run rate`) as avg_run_rate
FROM odi_2011_stats
WHERE `knockout yes or no` = 'no'
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT (`Team 1 batting run rate`) as avg_run_rate
FROM odi_2011_stats
WHERE `knockout yes or no` = 'yes'
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['avg_run_rate'], knockout_df['avg_run_rate'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

Index(['avg_run_rate'], dtype='object')
Index(['avg_run_rate'], dtype='object')
T-Statistic: 1.1521939913390382, P-Value: 0.26354330763997474


In [14]:
# Extract the data for non-knockout matches
query_non_knockout = """
SELECT (`Team 2 batting run rate`) as avg_run_rate2
FROM odi_2011_stats
WHERE `knockout yes or no` = 'no'
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT (`Team 2 batting run rate`) as avg_run_rate2
FROM odi_2011_stats
WHERE `knockout yes or no` = 'yes'
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['avg_run_rate2'], knockout_df['avg_run_rate2'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

Index(['avg_run_rate2'], dtype='object')
Index(['avg_run_rate2'], dtype='object')
T-Statistic: -0.4282363419208832, P-Value: 0.6748654202413835


In [15]:
from scipy.stats import ttest_ind

# Extract the data for non-knockout matches
query_non_knockout = """
SELECT `Team 1 50's`
FROM odi_2023_stats
WHERE `knockout yes or no` = 'no'
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT `Team 1 50's`
FROM odi_2023_stats
WHERE `knockout yes or no` = 'yes'
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['Team 1 50\'s'], knockout_df['Team 1 50\'s'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")


Index(['Team 1 50's'], dtype='object')
Index(['Team 1 50's'], dtype='object')
T-Statistic: 0.6053953260775005, P-Value: 0.5997721482788566


In [16]:
from scipy.stats import ttest_ind

# Extract the data for non-knockout matches
query_non_knockout = """
SELECT `Team 2 50's`
FROM odi_2023_stats
WHERE `knockout yes or no` = 'no'
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT `Team 2 50's`
FROM odi_2023_stats
WHERE `knockout yes or no` = 'yes'
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['Team 2 50\'s'], knockout_df['Team 2 50\'s'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

Index(['Team 2 50's'], dtype='object')
Index(['Team 2 50's'], dtype='object')
T-Statistic: 0.5334349496690809, P-Value: 0.5971040628865253


/opt/conda/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


In [17]:
# Extract the data for non-knockout matches
query_non_knockout = """
SELECT (`Team 1 batting average`) 
FROM odi_2011_stats
WHERE `knockout yes or no` = 'no'
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT (`Team 1 batting average`) 
FROM odi_2011_stats
WHERE `knockout yes or no` = 'yes'
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['Team 1 batting average'], knockout_df['Team 1 batting average'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

Index(['Team 1 batting average'], dtype='object')
Index(['Team 1 batting average'], dtype='object')
T-Statistic: 0.2119347660018187, P-Value: 0.8344685909784555


In [18]:
from scipy.stats import ttest_ind
import pandas as pd

# Extract the data for non-knockout matches
query_non_knockout = """
SELECT `Team 2 batting average`
FROM odi_2011_stats
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT `Team 2 batting average`
FROM odi_2023_stats
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Check for NaN values
print("Non-knockout NaN values:\n", non_knockout_df.isna().sum())
print("Knockout NaN values:\n", knockout_df.isna().sum())

# Drop NaN values for the t-test
non_knockout_clean = non_knockout_df.dropna()
knockout_clean = knockout_df.dropna()

# Ensure there are enough data points
print("Non-knockout clean data count:", len(non_knockout_clean))
print("Knockout clean data count:", len(knockout_clean))

if len(non_knockout_clean) > 1 and len(knockout_clean) > 1:
    # Perform the t-test
    t_stat, p_val = ttest_ind(non_knockout_clean['Team 2 batting average'], knockout_clean['Team 2 batting average'], equal_var=False)
    print(f"T-Statistic: {t_stat}, P-Value: {p_val}")
else:
    print("Not enough data to perform t-test.")


Index(['Team 2 batting average'], dtype='object')
Index(['Team 2 batting average'], dtype='object')
Non-knockout NaN values:
 Team 2 batting average    1
dtype: int64
Knockout NaN values:
 Team 2 batting average    0
dtype: int64
Non-knockout clean data count: 20
Knockout clean data count: 39
T-Statistic: -1.5598235038336565, P-Value: 0.12482042619343912


In [19]:
# Extract the data for non-knockout matches
query_non_knockout = """
SELECT (`Team 1 catches taken`) 
FROM odi_2011_stats
WHERE `knockout yes or no` = 'no'
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT (`Team 1 catches taken`) 
FROM odi_2011_stats
WHERE `knockout yes or no` = 'yes'
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['Team 1 catches taken'], knockout_df['Team 1 catches taken'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

Index(['Team 1 catches taken'], dtype='object')
Index(['Team 1 catches taken'], dtype='object')
T-Statistic: -0.363517165952748, P-Value: 0.7288457754021995


In [20]:
# Extract the data for non-knockout matches
query_non_knockout = """
SELECT (`Team 2 catches taken`) 
FROM odi_2011_stats
WHERE `knockout yes or no` = 'no'
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT (`Team 2 catches taken`) 
FROM odi_2011_stats
WHERE `knockout yes or no` = 'yes'
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['Team 2 catches taken'], knockout_df['Team 2 catches taken'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

Index(['Team 2 catches taken'], dtype='object')
Index(['Team 2 catches taken'], dtype='object')
T-Statistic: 0.7588599165480585, P-Value: 0.4578009160629263


In [21]:
# Extract the data for non-knockout matches
query_non_knockout = """
SELECT (`Team 1 runs scored by 4s`) 
FROM odi_2011_stats
WHERE `knockout yes or no` = 'no'
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT (`Team 1 runs scored by 4s`) 
FROM odi_2011_stats
WHERE `knockout yes or no` = 'yes'
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['Team 1 runs scored by 4s'], knockout_df['Team 1 runs scored by 4s'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

Index(['Team 1 runs scored by 4s'], dtype='object')
Index(['Team 1 runs scored by 4s'], dtype='object')
T-Statistic: 0.27286597671487206, P-Value: 0.7904579398876495


In [22]:
# Extract the data for non-knockout matches
query_non_knockout = """
SELECT (`Team 2 runs scored by 4s`) 
FROM odi_2011_stats
WHERE `knockout yes or no` = 'no'
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT (`Team 2 runs scored by 4s`) 
FROM odi_2011_stats
WHERE `knockout yes or no` = 'yes'
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['Team 2 runs scored by 4s'], knockout_df['Team 2 runs scored by 4s'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

Index(['Team 2 runs scored by 4s'], dtype='object')
Index(['Team 2 runs scored by 4s'], dtype='object')
T-Statistic: -1.1214813336868226, P-Value: 0.28187595939226456


In [23]:
# Extract the data for non-knockout matches
query_non_knockout = """
SELECT (`Team 1 runs scored by 6s`) 
FROM odi_2011_stats
WHERE `knockout yes or no` = 'no'
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT (`Team 1 runs scored by 6s`) 
FROM odi_2011_stats
WHERE `knockout yes or no` = 'yes'
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['Team 1 runs scored by 6s'], knockout_df['Team 1 runs scored by 6s'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

Index(['Team 1 runs scored by 6s'], dtype='object')
Index(['Team 1 runs scored by 6s'], dtype='object')
T-Statistic: 2.174683077569506, P-Value: 0.042598980257015466


In [24]:
# Extract the data for non-knockout matches
query_non_knockout = """
SELECT (`Team 2 runs scored by 6s`) 
FROM odi_2011_stats
WHERE `knockout yes or no` = 'no'
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT (`Team 2 runs scored by 6s`) 
FROM odi_2011_stats
WHERE `knockout yes or no` = 'yes'
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['Team 2 runs scored by 6s'], knockout_df['Team 2 runs scored by 6s'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

Index(['Team 2 runs scored by 6s'], dtype='object')
Index(['Team 2 runs scored by 6s'], dtype='object')
T-Statistic: -0.0731210359843199, P-Value: 0.943075228753395


In [25]:
from scipy.stats import ttest_ind

# Extract the data for non-knockout matches
query_non_knockout = """
SELECT `Team 1 50's`
FROM odi_2011_stats
WHERE `knockout yes or no` = 'no'
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT `Team 1 50's`
FROM odi_2011_stats
WHERE `knockout yes or no` = 'yes'
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['Team 1 50\'s'], knockout_df['Team 1 50\'s'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")


Index(['Team 1 50's'], dtype='object')
Index(['Team 1 50's'], dtype='object')
T-Statistic: 0.3668996928526713, P-Value: 0.7189561367469564


In [26]:
from scipy.stats import ttest_ind

# Extract the data for non-knockout matches
query_non_knockout = """
SELECT `Team 2 50's`
FROM odi_2011_stats
WHERE `knockout yes or no` = 'no'
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT `Team 2 50's`
FROM odi_2011_stats
WHERE `knockout yes or no` = 'yes'
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['Team 2 50\'s'], knockout_df['Team 2 50\'s'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

Index(['Team 2 50's'], dtype='object')
Index(['Team 2 50's'], dtype='object')
T-Statistic: -0.6069769786668839, P-Value: 0.5618664900272796


In [27]:
from scipy.stats import ttest_ind

# Extract the data for non-knockout matches
query_non_knockout = """
SELECT `Team 1 100's`
FROM odi_2011_stats
WHERE `knockout yes or no` = 'no'
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT `Team 1 100's`
FROM odi_2011_stats
WHERE `knockout yes or no` = 'yes'
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['Team 1 100\'s'], knockout_df['Team 1 100\'s'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

Index(['Team 1 100's'], dtype='object')
Index(['Team 1 100's'], dtype='object')
T-Statistic: 0.9390898592677526, P-Value: 0.3644187617737117


In [28]:
from scipy.stats import ttest_ind

# Extract the data for non-knockout matches
query_non_knockout = """
SELECT `Team 2 100's`
FROM odi_2011_stats
WHERE `knockout yes or no` = 'no'
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT `Team 2 100's`
FROM odi_2011_stats
WHERE `knockout yes or no` = 'yes'
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['Team 2 100\'s'], knockout_df['Team 2 100\'s'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

Index(['Team 2 100's'], dtype='object')
Index(['Team 2 100's'], dtype='object')
T-Statistic: -0.7844645405527361, P-Value: 0.46574501601592094


In [29]:
from pyspark.sql import SparkSession
import pandas as pd
from scipy.stats import ttest_ind

# Extract the data for the performance metric (e.g., Team 1 batting run rate) in 2011
query_2011 = """
SELECT `Team 2 batting average`
FROM odi_2011_stats
"""
team1_batting_run_rate_2011 = spark.sql(query_2011).toPandas()

# Extract the data for the performance metric (e.g., Team 1 batting run rate) in 2023
query_2023 = """
SELECT `Team 2 batting average`
FROM odi_2023_stats
"""
team1_batting_run_rate_2023 = spark.sql(query_2023).toPandas()

# Perform the t-test
t_stat, p_val = ttest_ind(team1_batting_run_rate_2011['Team 2 batting average'], team1_batting_run_rate_2023['Team 2 batting average'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

T-Statistic: nan, P-Value: nan


In [30]:
from pyspark.sql import SparkSession
import pandas as pd
from scipy.stats import ttest_ind

# Extract the data for the performance metric (e.g., Team 1 batting run rate) in 2011
query_2011 = """
SELECT `Team 2 batting run rate`
FROM odi_2011_stats
"""
team1_batting_run_rate_2011 = spark.sql(query_2011).toPandas()

# Extract the data for the performance metric (e.g., Team 1 batting run rate) in 2023
query_2023 = """
SELECT `Team 2 batting run rate`
FROM odi_2023_stats
"""
team1_batting_run_rate_2023 = spark.sql(query_2023).toPandas()

# Perform the t-test
t_stat, p_val = ttest_ind(team1_batting_run_rate_2011['Team 2 batting run rate'], team1_batting_run_rate_2023['Team 2 batting run rate'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

T-Statistic: -2.37020112877423, P-Value: 0.021658830257830854


In [31]:
# Extract the data for the performance metric (e.g., Team 1 batting run rate) in 2011
query_2011 = """
SELECT `Team 1 catches taken`
FROM odi_2011_stats
"""
team1_batting_run_rate_2011 = spark.sql(query_2011).toPandas()

# Extract the data for the performance metric (e.g., Team 1 batting run rate) in 2023
query_2023 = """
SELECT `Team 1 catches taken`
FROM odi_2023_stats
"""
team1_batting_run_rate_2023 = spark.sql(query_2023).toPandas()

# Perform the t-test
t_stat, p_val = ttest_ind(team1_batting_run_rate_2011['Team 1 catches taken'], team1_batting_run_rate_2023['Team 1 catches taken'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

T-Statistic: -1.8651604582825052, P-Value: 0.06738164739076269


In [32]:
# Extract the data for the performance metric (e.g., Team 1 batting run rate) in 2011
query_2011 = """
SELECT `Team 2 catches taken`
FROM odi_2011_stats
"""
team1_batting_run_rate_2011 = spark.sql(query_2011).toPandas()

# Extract the data for the performance metric (e.g., Team 1 batting run rate) in 2023
query_2023 = """
SELECT `Team 2 catches taken`
FROM odi_2023_stats
"""
team1_batting_run_rate_2023 = spark.sql(query_2023).toPandas()

# Perform the t-test
t_stat, p_val = ttest_ind(team1_batting_run_rate_2011['Team 2 catches taken'], team1_batting_run_rate_2023['Team 2 catches taken'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

T-Statistic: -2.9876391156453916, P-Value: 0.00459785189659269


In [33]:
# Extract the data for the performance metric (e.g., Team 1 batting run rate) in 2011
query_2011 = """
SELECT `Team 1 wickets taken by run out`
FROM odi_2011_stats
"""
team1_batting_run_rate_2011 = spark.sql(query_2011).toPandas()

# Extract the data for the performance metric (e.g., Team 1 batting run rate) in 2023
query_2023 = """
SELECT `Team 1 wickets taken by run out`
FROM odi_2023_stats
"""
team1_batting_run_rate_2023 = spark.sql(query_2023).toPandas()

# Perform the t-test
t_stat, p_val = ttest_ind(team1_batting_run_rate_2011['Team 1 wickets taken by run out'], team1_batting_run_rate_2023['Team 1 wickets taken by run out'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

T-Statistic: 1.3434424168850525, P-Value: 0.18880222995591356


In [34]:
# Extract the data for the performance metric (e.g., Team 1 batting run rate) in 2011
query_2011 = """
SELECT `Team 2 wickets taken by run out`
FROM odi_2011_stats
"""
team1_batting_run_rate_2011 = spark.sql(query_2011).toPandas()

# Extract the data for the performance metric (e.g., Team 1 batting run rate) in 2023
query_2023 = """
SELECT `Team 2 wickets taken by run out`
FROM odi_2023_stats
"""
team1_batting_run_rate_2023 = spark.sql(query_2023).toPandas()

# Perform the t-test
t_stat, p_val = ttest_ind(team1_batting_run_rate_2011['Team 2 wickets taken by run out'], team1_batting_run_rate_2023['Team 2 wickets taken by run out'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

T-Statistic: 1.689066839004371, P-Value: 0.10122236110158525


In [35]:
# Extract the data for the performance metric (e.g., Team 1 batting run rate) in 2011
query_2011 = """
SELECT `Team 1 - runs scored by extras`
FROM odi_2011_stats
"""
team1_batting_run_rate_2011 = spark.sql(query_2011).toPandas()

# Extract the data for the performance metric (e.g., Team 1 batting run rate) in 2023
query_2023 = """
SELECT `Team 1 - runs scored by extras`
FROM odi_2023_stats
"""
team1_batting_run_rate_2023 = spark.sql(query_2023).toPandas()

# Perform the t-test
t_stat, p_val = ttest_ind(team1_batting_run_rate_2011['Team 1 - runs scored by extras'], team1_batting_run_rate_2023['Team 1 - runs scored by extras'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

T-Statistic: 0.4087692912369593, P-Value: 0.6843478455579354


In [36]:
# Extract the data for the performance metric (e.g., Team 1 batting run rate) in 2011
query_2011 = """
SELECT `Team 2 - runs scored by extras`
FROM odi_2011_stats
"""
team1_batting_run_rate_2011 = spark.sql(query_2011).toPandas()

# Extract the data for the performance metric (e.g., Team 1 batting run rate) in 2023
query_2023 = """
SELECT `Team 2 - runs scored by extras`
FROM odi_2023_stats
"""
team1_batting_run_rate_2023 = spark.sql(query_2023).toPandas()

# Perform the t-test
t_stat, p_val = ttest_ind(team1_batting_run_rate_2011['Team 2 - runs scored by extras'], team1_batting_run_rate_2023['Team 2 - runs scored by extras'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

T-Statistic: 1.1173735925231396, P-Value: 0.2712465327968689


In [37]:
from scipy.stats import ttest_ind

# Extract the data for non-knockout matches
query_non_knockout = """
SELECT `Team 1 50's`
FROM odi_2023_stats
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT `Team 1 50's`
FROM odi_2011_stats
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['Team 1 50\'s'], knockout_df['Team 1 50\'s'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

Index(['Team 1 50's'], dtype='object')
Index(['Team 1 50's'], dtype='object')
T-Statistic: 0.9857982454492672, P-Value: 0.329952913078409


In [38]:
from scipy.stats import ttest_ind

# Extract the data for non-knockout matches
query_non_knockout = """
SELECT `Team 2 50's`
FROM odi_2023_stats
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT `Team 2 50's`
FROM odi_2011_stats
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['Team 2 50\'s'], knockout_df['Team 2 50\'s'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

Index(['Team 2 50's'], dtype='object')
Index(['Team 2 50's'], dtype='object')
T-Statistic: -0.07027236201927475, P-Value: 0.9443488762808223


In [39]:
from scipy.stats import ttest_ind

# Extract the data for non-knockout matches
query_non_knockout = """
SELECT `Team 1 100's`
FROM odi_2023_stats
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT `Team 1 100's`
FROM odi_2011_stats
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['Team 1 100\'s'], knockout_df['Team 1 100\'s'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

Index(['Team 1 100's'], dtype='object')
Index(['Team 1 100's'], dtype='object')
T-Statistic: 0.7143824566430605, P-Value: 0.47838804375863586


In [40]:
# Extract the data for non-knockout matches
query_non_knockout = """
SELECT `Team 2 100's`
FROM odi_2023_stats
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT `Team 2 100's`
FROM odi_2011_stats
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['Team 2 100\'s'], knockout_df['Team 2 100\'s'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

Index(['Team 2 100's'], dtype='object')
Index(['Team 2 100's'], dtype='object')
T-Statistic: 1.0121587269618053, P-Value: 0.31665098877512754


In [41]:
# Extract the data for non-knockout matches
query_non_knockout = """
SELECT `team 1 % of overs bowled by pace`
FROM odi_2023_stats
"""
non_knockout_df = spark.sql(query_non_knockout).toPandas()

# Extract the data for knockout matches
query_knockout = """
SELECT `team 1 % of overs bowled by pace`
FROM odi_2011_stats
"""
knockout_df = spark.sql(query_knockout).toPandas()

# Print column names to verify
print(non_knockout_df.columns)
print(knockout_df.columns)

# Perform the t-test
t_stat, p_val = ttest_ind(non_knockout_df['team 1 % of overs bowled by pace'], knockout_df['team 1 % of overs bowled by pace'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")


Index(['team 1 % of overs bowled by pace'], dtype='object')
Index(['team 1 % of overs bowled by pace'], dtype='object')
T-Statistic: 0.4116019745485165, P-Value: 0.6829297593672659


In [42]:
# Extract the data for the performance metric 
query_2011 = """
SELECT `team 1 % of overs bowled by pace`
FROM odi_2011_stats
"""
team1_batting_run_rate_2011 = spark.sql(query_2011).toPandas()

# Extract the data for the performance metric 
query_2023 = """
SELECT `team 1 % of overs bowled by pace`
FROM odi_2023_stats
"""
team1_batting_run_rate_2023 = spark.sql(query_2023).toPandas()

# Perform the t-test
t_stat, p_val = ttest_ind(team1_batting_run_rate_2011['team 1 % of overs bowled by pace'], team1_batting_run_rate_2023['team 1 % of overs bowled by pace'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

T-Statistic: -0.4116019745485165, P-Value: 0.6829297593672659


In [43]:
# Extract the data for the performance metric 
query_2011 = """
SELECT `team 2 % of overs bowled by pace`
FROM odi_2011_stats
"""
team1_batting_run_rate_2011 = spark.sql(query_2011).toPandas()

# Extract the data for the performance metric 
query_2023 = """
SELECT `team 2 % of overs bowled by pace`
FROM odi_2023_stats
"""
team1_batting_run_rate_2023 = spark.sql(query_2023).toPandas()

# Perform the t-test
t_stat, p_val = ttest_ind(team1_batting_run_rate_2011['team 2 % of overs bowled by pace'], team1_batting_run_rate_2023['team 2 % of overs bowled by pace'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

T-Statistic: -3.059410028955035, P-Value: 0.004855469185088766


In [44]:
# Extract the data for the performance metric 
query_2011 = """
SELECT `team 1% of overs bowled by spin`
FROM odi_2011_stats
"""
team1_batting_run_rate_2011 = spark.sql(query_2011).toPandas()

# Extract the data for the performance metric 
query_2023 = """
SELECT `team 1% of overs bowled by spin`
FROM odi_2023_stats
"""
team1_batting_run_rate_2023 = spark.sql(query_2023).toPandas()

# Perform the t-test
t_stat, p_val = ttest_ind(team1_batting_run_rate_2011['team 1% of overs bowled by spin'], team1_batting_run_rate_2023['team 1% of overs bowled by spin'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

T-Statistic: 0.4116019745485165, P-Value: 0.6829297593672659


In [45]:
# Extract the data for the performance metric in 2011
query_2011 = """
SELECT `team 2 -3s`
FROM odi_2011_stats
"""
team2_wickets_pace_2011 = spark.sql(query_2011).toPandas()

# Extract the data for the performance metric in 2023
query_2023 = """
SELECT `team 2 -3s`
FROM odi_2023_stats
"""
team2_wickets_pace_2023 = spark.sql(query_2023).toPandas()

# Perform the t-test
t_stat, p_val = ttest_ind(team2_wickets_pace_2011['team 2 -3s'], team2_wickets_pace_2023['team 2 -3s'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")


T-Statistic: 2.8591942909317645, P-Value: 0.0074707783318563896


In [46]:
# Extract the data for the performance metric in 2011
query_2011 = """
SELECT `team 2-  number of players  in playing 11 who have played  in franchise leagues during this tournament`
FROM odi_2011_stats
"""
team2_wickets_pace_2011 = spark.sql(query_2011).toPandas()

# Extract the data for the performance metric in 2023
query_2023 = """
SELECT `team 2-  number of players  in playing 11 who have played  in franchise leagues during this tournament`
FROM odi_2023_stats
"""
team2_wickets_pace_2023 = spark.sql(query_2023).toPandas()

# Perform the t-test
t_stat, p_val = ttest_ind(team2_wickets_pace_2011['team 2-  number of players  in playing 11 who have played  in franchise leagues during this tournament'], team2_wickets_pace_2023['team 2-  number of players  in playing 11 who have played  in franchise leagues during this tournament'], equal_var=False)

print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

T-Statistic: -5.2097673464698255, P-Value: 2.54934209659638e-05


In [47]:
# Calculate percentage change for IPL run rate from 2010 to 2022
ipl_percentage_change = ((total_amount_of_catches_taken - ipl_2010_run_rate) / ipl_2010_run_rate) * 100

# Calculate percentage change for ODI run rate from 2011 to 2023
odi_percentage_change = ((team2_batting_run_rate_2023.mean() - team2_batting_run_rate_2011.mean()) / team2_batting_run_rate_2011.mean()) * 100

ipl_percentage_change, odi_percentage_change

NameError: name 'total_amount_of_catches_taken' is not defined